# pip install this
0 to PyPI in 60 minutes

In [1]:
! ./cleanup.sh

import os
os.chdir('hippy-chat-stuff')

## Introduction
It's a great time to be a developer! There are so many great languages, lots of cool emerging tools, tons of documentation, and it's all so easy to get!

The open-source community is really flourishing in the Python universe. But, getting your software published can seem like a very daunting task to new developers (and even veterans that haven't published before).

Lot of tools are emerging that make publishing your software easier with a lot less 'fiddling'. Removing the barrier of tedium should encourage more diversity (in all senses of the word) in our code ecosystem and developer community



## About this talk
There's a lot of material to cover here, and the material is discussion worthy.

However, to get through everything I have prepared here, I have to ask for questions to be held until the end.

That being said, I look forward to discussing details, personal experiences, and individual perspectives when we get to the end of the talk.

### Disclaimers
Python is an opinionated language. I'm an opinionated developer. This is fine, as long as it is understood.

I prefer Python 3. This talk is in python 3. That's all I have to say about that

Holy wars are fought over dogma, not philosophy. So, start with [The Zen of Python]( https://www.python.org/dev/peps/pep-0020/) and then try to find community accepted best-practices from there. 

Sometimes, it's better to accept a convention you don't personally like for the sake of consistency and accessibility.



### Other Thoughts
Syntax checking is great! It takes the thinking out of code formatting. So, I like to use vanilla pep8 via out-of-the-box [flake8](https://pypi.python.org/pypi/flake8)

There is a lot of philosophy and discussion about packaging. Python software is most commonly distributed in packages on [PyPI](https://pypi.python.org/pypi), but there are alternative packagers, repositories, and schools of thought. We're not going to go into this here, but it is interesting! Instead, this talk is intended to be a pragmatic guide to getting your code published.

I've focused on simple and easy tools and procedures. Some of the things I talk about will be personal best practices. I'll call these out with _italics_. Your mileage may vary, and I understand and hope you have your own set of best-practices or preferences. I'm happy to talk about them later.

## Stuff to talk about if I have enough time
* pip vs easy_install
* conda. Supposed to be really good...haven't tried it yet


## Starting from not quite 0
At comScore, we use Hipchat for team communication. I wanted to see critical errors in my team's hipchat channel (I'm lead dev...I get to decide that =P)

There weren't any extensions for python logging that do this yet, so I rolled my own using the very cool [Hipchat API](https://www.hipchat.com/docs/apiv2)

Since it's a useful tool, and there is nothing proprietary in it, I think it would be a good thing to get out into the Python OSS community.

So, in this talk, we're going to publish this logging handler as an independent python package


## The basic python package

There are, again, a lot of opinions about best-practices in structuring python packages. What I'm presenting here is a combination of best-practices, easy and convenient methodologies, and my peferred packages.

The package structure for our package we're building today is as follows:
```
hippy-chat/
|--> setup.py
|--> MANIEST.in
|--> CHANGELOG.rst
|--> README.rst
|--> .project_metadata.json
|--> .hipchat.json
|--> hippy_chat
     |--> __init__.py
     |--> handler.py
     |--> exec/
          |--> demo.py
```

* setup.py provides info to both install **and** build your package
* Name of the root directory doesn't actually matter
* Don't put metadata in the base \__init\__.py
* Only recently started putting executables in exec/
* I'll talk about .project_metadata.json later. This is a _personal peference_
* I use ReStructuredText for my documents, because it's what sphinx uses. _I also like it better than markdown_

## Support files

### The setup.py file for the project
Some things to note:
* I use the setup method from setuptools instead of distutils
* But, _I like to install with pip even locally_
* So, none of this stuff is tested using ```$ python setup.py install```
* I've used a lot of variations of setup.py structure, and this is the layout I've arrived at.
* I promise you I'm not done trying new things!

In [2]:
! cat setup.py

import json
import setuptools


with open('.project_metadata.json') as meta_file:
    project_metadata = json.loads(meta_file.read())


setuptools.setup(
    name=project_metadata['name'],
    version=project_metadata['release'],
    author=project_metadata['author'],
    author_email=project_metadata['author_email'],
    description=project_metadata['description'],
    license=project_metadata['license'],
    install_requires=[
        'arrow',
        'click',
        'py-buzz',
        'requests',
    ],
    extras_require={
        'dev': [
            'flake8',
            'pytest',
            'pytest-capturelog',
        ],
    },
    include_package_data=True,
    packages=setuptools.find_packages(),
    entry_points={
        'console_scripts': [
            'hippy-chat-demo = hippy_chat.exec.demo:main',
        ],
    },
)


#### Why .project_metadata.json ???
```
    version=project_metadata['release'],
```
There's [lots and lots of discussion](https://www.google.com/search?q=python+best+practices+project+metadata) about how to include meta-data (especially version data) in your project. There doesn't seem to be a universally accepted best practice, so I came up with my own.

Keeing project metadata in the root project folder this way has some advantages:
* json. Everything can parse json.
* dynamic. I can change metadata (like version number) in one place
* other stuff can easily find it. Like sphinx, for instance (we'll see this later)

#### Why extras_require?
```
    extras_require={
        'dev': [
            'flake8',
            'pytest',
            'pytest-capturelog',
        ],
    },
```
Using the extras_require directive lets you have differing requirements for different roles. Basic installs shouldn't install dependencies that are only needed for dev, testing, document generation, etc.

To install using a different role, you can do it like this:
```
$ pip install <package>[dev]
```




#### Why include_package_data?
```
    include_package_data=True,
```

You'll need to include this to also install stuff like .project_metadata.json or any other files not a part of your source tree. These will need to be specified in MANIFEST.in.

**Note**: If you're reading through examples and/or the distutils/setuptools specification, you might find (and be rather tempted by) ```data_files=...```. I have not found this to work consistently between distutils, setuptools, and pip.

#### Why find_packages?
Because it's smart and really easy. Otherwise, you have to list your main package, and all sub-packages therein:
```
    packages=['a', 'a.a', 'a.a.a', 'a.b', 'a.b.a']
```
If you add additional sub-packages, you will have to remember to add them here. Of course, you won't notice until you publish to PyPI and try to pip install your package next time...


#### Why entry_points?
I only started doing this recently.

Previously, I kept all my executables in ```<root>/bin/``` and added a ```scripts=glob.blog('bin/*')```. This works ok, but there are a few things that make it frustrating
* No (good) way to import the scripts for unit testing
* Don't forget to add your shebang!
* Make sure your shebang calls out the right python version
* Make sure that your executables have the executable flag set
* Probably shouldn't name them with an extension (as is usual in unix for executable files)
* Not cross-platform

Using entry points gives you a lot of magic that takes care of those things (for the most part). Pip will create the executable in a platform specific way at install time, rename it, and make sure that it references the right python interpreter. Further, since the executables are part of the package itself, you can easily unit test them. Finally, a single module can have multiple entry points that are all split out into their own executables!

It should also be noted that this pattern works really well with click.

### The CHANGELOG.rst file
Changelogs are an important way for users to track what has changed between releases. This way, they can peruse major differences and decide if upgrading is worth the risk to their software. There is some really great discussion about this topic provided at [keepachagnelog.com](http://keepachangelog.com/en/0.3.0/). After reading through, I became a believer.

### The MANIFEST.in file
As stated above, this is the most consistent way to make sure that extra files are installed with your project. The file structure is pretty much dead simple. Just list exactly the extra files that you want to include

In [3]:
! cat MANIFEST.in

include .project_metadata.json


### The README.rst file
You really should have a README immediately. Notice no italics? This isn't an opinion. It's pretty important. Your README doesn't have to be really verbose. It also probably shouldn't be the difinitive documentation for your project (you should build those with sphinx...but more on that later). However, the README should be a touchstone for people who want to use your package. At a minimum it should include
* A summary, no matter how terse, of what your project is for and what it does
* Basic requirements. **Especially** which versions of python it supports
* An explanation of how to get an environment set up for development and testing

In [4]:
! cat README.rst

************
 hippy-chat
************

----------------------------------
Hipchat logging handler for python
----------------------------------

This package provides a logging handler that can send messages to a hipchat
room in the form of notifications.

Requirements

 - Python 3

Installing
Install using pip::

$ pip install hippy-chat

Using
=====
Setup the handler with the needed credentials for the hipchat server's api that
you will be sending messages to:

.. code-block:: python

   from hippy_chat.handler import HipchatHandler

   handler = HipchatHandler(
       'https://some_hipchat_server', 'some-room', 'SOME_HIPCHAT_API_TOKEN',
   )
   logger.addHandler(handler)

Contributing

In order to get your environment set up for development, it is recommended to
use virtualenv and pip to install dev dependencies::

    $ virtualenv --python=python3 env
    $ source env/bin/activate
    $ pip install -e .[dev]


### The .project_metadata.json file
As I explained above, this is a personal best-practice. However you do it, though, you should try to keep your project metadata together in one and only one place. This is especially important for project versions. You don't want to confuse your user when they see one version on your documentation and another on the package they download from PyPI. Using a single metadata file like this can help with these sorts of problems

In [5]:
! cat .project_metadata.json

{
    "author": "Tucker Beck",
    "author_email": "tucker.beck@gmail.com",
    "copyright": "2017, Tucker Beck",
    "description": "Log messages to a hipchat room",
    "license": "MIT",
    "name": "hippy-chat",
    "release": "0.1.0",
    "version": "0.1"
}


In [6]:
root_dir = os.path.normpath('../../hippy-chat')
! mkdir $root_dir
for filename in [
    'setup.py', 'README.rst', 'MANIFEST.in', 'CHANGELOG.rst',
    '.project_metadata.json', '.hipchat.json',
]:
    ! cp $filename $root_dir

## Source Code
There are a few best practices with how source code should be laid out:

* All source should go in a directory in the project root
* This directory should be in lower **snake_case**, and should be some derivation of your package name
* You should try to pick something unique enough that it won't collide with other packages 
* This directory name will be the importable name of your package
  
There are, of course, a lot of others. But these are the basics
  

### The hippy_chat handler module
This is the source for our chat handler. A few things to note:
* [arrow](http://crsmithdev.com/arrow/) is a really nice datetime replacement. _I recommend using this or [pendulum](https://github.com/sdispater/pendulum) for **all** cases where you work with datetimes_
* [py-buzz](https://github.com/dusktreader/py-buzz) is one of my packages that provides some nice add-ons to python exceptions
* [requests](http://docs.python-requests.org/en/master/) is an awesome package for working with apis. It's dead simple to use.

Also worth noting: This is a logging handler for normal python logging. This is what our project started using a long time ago. Python logging is pretty cool, but _it is not as cool as it could or should be. These days, I try to use [logbook](https://github.com/getlogbook/logbook) wherever possible_. However, logbook is not a drop-in replacement, so our project will probably stick with vanilla logging for now...or maybe forever

In [7]:
! cat handler.py

import arrow
import buzz
import json
import logging
import requests
import textwrap
import urllib


HIPCHAT_DEFAULT_TITLE = 'event occured in application'


class HipchatHandler(logging.Handler):
    """
    This module provides a logging handler that sends notifications to a
    hipchat room. The notifications will appear as expandable 'cards'
    with relevant data.

    .. todo:: Add sentry support for the card urls
    """

    def __init__(
            self, url, room, token, title=HIPCHAT_DEFAULT_TITLE,
            *args, **kwargs
    ):
        """
        Initializes the handler

        :param: url:   The base URL to the hipchat server
        :param: room:  The name of the room in which to make the notification
        :param: token: The auth token for the room
        :param: title: An optional title to include in the notification
        """

        self.url = '{url}/v2/room/{room}/notification'.format(
            url=url,
            room=urllib.parse.quote(room),
      

In [8]:
source_dir = os.path.join(root_dir, 'hippy_chat')
! mkdir $source_dir
! cp handler.py $source_dir

We will also need to add an empty __init__.py module. This tells Python that this directory contains a package and, potentially, subpackages. Python won't be able to find the 'handlers' module without this

In [9]:
! touch $source_dir/__init__.py

## Test Code
There are a few different python testing frameworks out there.

The unittest package is a part of the python standard library, and it offers a lot of features out of the box. However, I don't recommend it for a few reasons:
* Test discovery is painful
* Using the special assert methods requires a lot of lookup time that could be spent writing tests
* Everything **has** to be done in test classes and heirarchies thereof
* No pluggable extensions

For a long time, [nose](http://nose.readthedocs.io/en/latest/) addressed a lot of those concerns. However, nose is no longer under active development, and, its replacement, [nose2](https://github.com/nose-devs/nose2) is still in 'beta' (pre 1.0 release) and is not progressing very rapidly. Additionally, _nose introduces some of its own quirks that can be annoying_.

I find that [pytest](http://doc.pytest.org/en/latest/) is a great testing framework that allows you to get testing very rapidly and yet hass enough features to support basically anything you want to do. There are a lot of useful pytest plugins out there as well to fill in missing functionality.

The pytest framework _can_ use test class heirarchies, but they are optional. Most setup in pytest is done via fixtures, which can be incredibly powerful (if somewhat confusing at times)

Project wide and directory wide configuration is available via `conftest.py` files, but we won't need any for hippy-chat, and you may not need them for your project

Tests can either be
* interleaved with your source code
* put in a sub-module of your source module
* put in their own directory

The most common practice, and my preferred approach, is to put the tests in their own directory immediately below the root of your project. This makes them very easy to target with your command line and also lowers the risk of accidental imports.

### The test_handler.py test module
This module contains the unit tests for the hippy_chat.handler module. This example is pretty spartan, but there should be enough here to show some pytest features.


In [10]:
! cat test_handler.py

import collections
import json
import logging
import pytest
import requests

from hippy_chat.handler import HipchatHandler, HIPCHAT_DEFAULT_TITLE


class TestHipchatHandler:

    @pytest.fixture(autouse=True)
    def setup(self):
        """
        Sets up the logger for the tests in this class. Is automatically
        invoked before each test function is called
        """
        self.logger = logging.getLogger('TestHipchatLogger')
        self.logger.setLevel(logging.DEBUG)

    @pytest.yield_fixture
    def dummy_handler(self):
        """
        Provides a dummy instance of a HipchatHandler
        """
        dummy_url = 'https://some.fake.server.hipchat.com'
        dummy_room_name = 'some-room'
        dummy_token = 'SOME_TOKEN'

        dummy_handler = HipchatHandler(dummy_url, dummy_room_name, dummy_token)
        self.logger.addHandler(dummy_handler)
        yield dummy_handler
        self.logger.removeHandler(dummy_handler)

    def test___init__(self, dummy_handler):
 

In [11]:
test_dir = os.path.join(root_dir, 'tests')
! mkdir $test_dir
! cp test_handler.py $test_dir

Invoking pytest against our project is incredibly simple. You simply need to call it and target the test directory. You can target specific tests, test directories, or tests matching a specific pattern.

We'll invoke tests in a bit after we are done setting up the package

## Executables
Executable code should be as compact as possible. It's good practice to make your entry-points as concise as possible and have most logic within your python modules. Ideally, an executable does little more than parse a command line, call into a function, and then possibly produce output

I use [click](https://github.com/pallets/click) for command line processing. _This is a matter of personal preference_. However, you should **never do your own command line processing**. Even for really simple stuff, I reccommend using a command line parser. [argparse](https://docs.python.org/3/library/argparse.html) (and its predecessor, [optparse](https://docs.python.org/2/library/optparse.html)) is built into the language and provides a full-featured command-line parser.

### The demo.py executable file
This module will just be used to show a demonstration of the logging handler in action. It parses some basic command line options and does some logging to show the handler working

In [12]:
! cat demo.py

import arrow
import click
import json
import logging
import os
import time
import hippy_chat.handler


@click.command()
@click.option(
    '-c', '--config-file',
    default='.hipchat.json',
    help="the file containing hipchat deets",
)
@click.option(
    '-l', '--log-level',
    default='ERROR',
    type=click.Choice(['DEBUG', 'INFO', 'WARNING', 'ERROR', 'CRITICAL']),
    help="select the level of messges that should be logged to the room",
)
@click.option(
    '-i', '--message-interval',
    default=5,
    type=int,
    help="How often to message the room",
)
def main(config_file, log_level, message_interval):

    log_level = getattr(logging, log_level)

    logger = logging.getLogger(__name__)
    logger.setLevel(logging.INFO)

    with open(os.path.expanduser('.hipchat.json')) as config_file:
        config = json.load(config_file)
    hipchat_handler = hippy_chat.handler.HipchatHandler(**config)
    hipchat_handler.setLevel(log_level)

    logger.addHandler(hipchat_handler)

  

In [13]:
exec_dir = os.path.join(source_dir, 'exec')
! mkdir $exec_dir
! cp demo.py $exec_dir
! touch $exec_dir/__init__.py

## Working within a virtual environment
Python's virtual environments are just too nice to _not_ use. They are really simple to set up, and provide a good way to isolate your environment as you develop. Using virtual environments has the following benefits:
* Improve repeatability in package setup
* Easy _removal_ of your installed package
* Provide custom configuration to your package without breaking anything else
* Use whatever version of python interpreter you want

### Setting up the virtual environment
Note that I'm making the wild assumption that you have [virtualenv](https://virtualenv.pypa.io/en/stable/) set up. If not, the setup is pretty easy. That does, however, depend on your os

Creating and activating the virtualenv is pretty easy:

In [14]:
os.chdir(root_dir)

! virtualenv --python=python3 env
! source env/bin/activate

Running virtualenv with interpreter /Users/tbeck/.virtualenvs/pip-install-this/bin/python3
Using real prefix '/usr/local/Cellar/python3/3.5.1/Frameworks/Python.framework/Versions/3.5'
New python executable in /Users/tbeck/work/hippy-chat/env/bin/python3
Also creating executable in /Users/tbeck/work/hippy-chat/env/bin/python
Installing setuptools, pip, wheel...done.


Now invoking python (or a script that invokes python via a shebang line) in the active virtual environment will be working in its own encapsulated environment. This also means that any packages that are installed via pip will be localized to this virtual environment

## Installing the new package in 'edit' mode
While you are actively developing a project, it is very convenient to have it installed inside of your active virtual environment in 'editable' mode. Pip offers this feature so that you can modify a source code and see the changes reflected the next time you invoke python or a python script without having to re-install the package. Trust me, you want to use this feature when you are working on your package

Here we will invoke the install command against the pwd with the 'dev' modifier. This will install all of the normal package dependencies but will also install the dependencies from the 'dev' section of 'extras_require' in our setup.py:

In [15]:
! pip install -e .[dev]

Obtaining file:///Users/tbeck/work/hippy-chat
  Found existing installation: hippy-chat 0.1.0
    Uninstalling hippy-chat-0.1.0:
      Successfully uninstalled hippy-chat-0.1.0
  Running setup.py develop for hippy-chat


Now our environment has all of our dependencies installed in adition to the hippy-chat package that we are building

## Invoking the test suite

The pytest framework has a whole slew of options that you can specify on the command line. Two options I find most useful are the --maxfail and --verbose options

I generally invoke pytest against my entire test suite using the `--maxfail=1` option. This tells pytest to stop immediately upon the first test failure. This is useful for larger projects where you want to see a problem right away when a test fails. You usually won't set this if you are, for instance, running your test suite as a deployment step or any time you want a report of all failing tests

You can also increase the verbosity of failure messages by passing `--verbose` flags. The flags are additive, so two verbose flags will show even more verbose output.

Note: As of this writing, I discovered some weirdness with python interpreter and py.test vs pytest

In [16]:
! py.test --maxfail=1 --verbose tests/

============================= test session starts ==============================
platform darwin -- Python 3.5.1, pytest-3.0.6, py-1.4.32, pluggy-0.4.0 -- /Users/tbeck/.virtualenvs/pip-install-this/bin/python3.5
cachedir: .cache
rootdir: /Users/tbeck/work/hippy-chat, inifile: 
plugins: capturelog-0.7
collected 3 items 

tests/test_handler.py::TestHipchatHandler::test___init__ PASSED
tests/test_handler.py::TestHipchatHandler::test__make_payload PASSED
tests/test_handler.py::TestHipchatHandler::test_integration PASSED

============================ pytest-warning summary ============================
WI1 /Users/tbeck/.virtualenvs/pip-install-this/lib/python3.5/site-packages/pytest_capturelog.py:171 'pytest_runtest_makereport' hook uses deprecated __multicall__ argument
WC1 None pytest_funcarg__caplog: declaring fixtures using "pytest_funcarg__" prefix is deprecated and scheduled to be removed in pytest 4.0.  Please remove the prefix and use the @pytest.fixture decorator instead.
WC1 None p

## Trying out the demo
Now that everything is setup, we should be able to try out the demo. Note, that it is installed into our editable virtualenv, and is invokable by its installed name: 'hippy-chat-demo'

We'll watch for the results in [a hipchat room I set up for this talk](https://pip-install-this.hipchat.com/chat)

In [17]:
! hippy-chat-demo --log-level WARNING --message-interval=2